# Elderly Historical Destination Count
#### Daniel Fay

In [135]:
# Import necessary packages
import pandas as pd
import geopandas as gpd
import numpy as np

In [136]:
# Load data
trip_survey = pd.read_csv('../../fm-lm-demand-simulation/data/processed/trip_survey_working_shape.csv', index_col=0)
trip_survey = gpd.GeoDataFrame(trip_survey)
trip_survey.head()

,SAMPN,PERNO,PERTYPE,ORIG_HOME,DEST_HOME,DOW_x,OTAZ,DTAZ,OTPURP_AGG,DTPURP_AGG,...,CDRIV,WSTRT,WEND,STRVR,ENDVR,STUDE,TAZ_ID1,geometry,work_flex_start,work_flex_end
0,3000056,1,2,1,0,3.0,2626,2627,0,9,...,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No,2626.0,"POLYGON ((-73.4243 41.4551, -73.42529999999999...",0,1
1,3000056,1,2,1,0,3.0,2626,2627,0,1,...,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No,2626.0,"POLYGON ((-73.4243 41.4551, -73.42529999999999...",0,1
2,3000056,1,2,1,0,3.0,2626,2628,0,9,...,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No,2626.0,"POLYGON ((-73.4243 41.4551, -73.42529999999999...",0,1
3,3000056,2,1,1,0,3.0,2626,2676,0,1,...,No,730,1900,Within 15 Minutes or Less,More than 1 Hour,No,2626.0,"POLYGON ((-73.4243 41.4551, -73.42529999999999...",0,1
4,3000056,3,3,1,0,3.0,2626,2627,0,8,...,No,1800,2330,More than 1 Hour,More than 1 Hour,Yes - Full Time,2626.0,"POLYGON ((-73.4243 41.4551, -73.42529999999999...",1,1


In [137]:
# Load census tracts
census_tracts = gpd.read_file('../../fm-lm-demand-simulation/data/external/NYMTC_BPM_TAZ2010_CT2010/BPM_CT2010.shp')
census_tracts = census_tracts[['GEOID10', 'geometry']]


In [138]:
# Select only elderly samples
trip_survey_elderly = trip_survey.loc[trip_survey['AGE_R'] == '55+']
len(trip_survey_elderly)

38314

In [139]:
# Find count of each destination
elderly_dest_count = trip_survey_elderly.groupby('DTRACT').count()
elderly_dest_count = elderly_dest_count[['SAMPN']]
elderly_dest_count.columns = ['Count']
elderly_dest_count.reset_index(inplace=True)
elderly_dest_count.DTRACT = elderly_dest_count.DTRACT.astype(int)
elderly_dest_count.DTRACT = elderly_dest_count.DTRACT.astype(str)
elderly_dest_count.DTRACT = elderly_dest_count.DTRACT.apply(lambda x: x.zfill(11))
elderly_dest_count.head()

,DTRACT,Count
0,00000000000,1
1,09001010102,2
2,09001010201,2
3,09001010202,7
4,09001010300,5


In [140]:
len(elderly_dest_count)

4322

In [141]:
# Merge in geometry
elderly_dest_count = pd.merge(elderly_dest_count, census_tracts, left_on='DTRACT', right_on='GEOID10')
len(elderly_dest_count)

4148

In [143]:
elderly_dest_count = elderly_dest_count[['DTRACT', 'Count', 'geometry']]
elderly_dest_count.head()

,DTRACT,Count,geometry
0,09001010102,2,"POLYGON ((-73.599163 41.09901800000002, -73.59..."
1,09001010201,2,POLYGON ((-73.60366500000001 41.07464400000001...
2,09001010202,7,"POLYGON ((-73.568271 41.05947400000001, -73.56..."
3,09001010300,5,POLYGON ((-73.61725400000002 41.03698699999998...
4,09001010500,11,POLYGON ((-73.62720400000003 41.02118000000004...


In [148]:
elderly_dest_count = gpd.GeoDataFrame(elderly_dest_count)

In [149]:
nyc = gpd.read_file('../Data/shapefiles/nybb.shp')

In [157]:
elderly_dest_count.crs = {'init' :'epsg:4326'}
elderly_dest_count.crs

{'init': 'epsg:4326'}

In [159]:
nyc = nyc.to_crs({'init' :'epsg:4326'})

In [160]:
elderly_dest_count_nyc = gpd.sjoin(elderly_dest_count, nyc, op='within')

In [162]:
elderly_dest_count_nyc.to_file('../Data/shapefiles/elderly_dest_count.shp')